## Optimize the Model

In [23]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [24]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [25]:
# Drop the non-beneficial ID columns

application_df = application_df.drop(["EIN"],1)

C:\Users\thong\AppData\Local\Temp\ipykernel_16152\521640425.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  application_df = application_df.drop(["EIN"],1)


In [26]:
application_df.nunique()

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [27]:
# Look at NAME value counts for binning
name_counts = application_df['NAME'].value_counts()
#  How many name counts are greater than 5?
name_counts[name_counts>5]

PARENT BOOSTER USA INC                             1260
TOPS CLUB INC                                       765
UNITED STATES BOWLING CONGRESS INC                  700
WASHINGTON STATE UNIVERSITY                         492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC     408
                                                   ... 
OLD OAK CLIFF CONSERVATION LEAGUE INC                 6
AMERICAN NEPHROLOGY NURSES ASSOCIATION                6
HUMBLE ISD EDUCATIONAL SUPPORT GROUPS INC             6
PROFESSIONAL LOADMASTER ASSOCIATION                   6
CBMC INC                                              6
Name: NAME, Length: 354, dtype: int64

In [28]:
# Determine which values to replace if counts are less than or equal to 5.
names_to_replace = list(name_counts[name_counts <= 5].index)

# Replace in dataframe
for app in names_to_replace:
    application_df['NAME'] = application_df['NAME'].replace(app,"Other")

# Check to make sure binning was successful
application_df['NAME'].value_counts()

Other                                           20043
PARENT BOOSTER USA INC                           1260
TOPS CLUB INC                                     765
UNITED STATES BOWLING CONGRESS INC                700
WASHINGTON STATE UNIVERSITY                       492
                                                ...  
HABITAT FOR HUMANITY INTERNATIONAL                  6
DAMAGE PREVENTION COUNCIL OF TEXAS                  6
FLEET RESERVE ASSOCIATION                           6
HUGH OBRIAN YOUTH LEADERSHIP                        6
INTERNATIONAL CONGRESS OF CHURCHES MINISTERS        6
Name: NAME, Length: 355, dtype: int64

In [29]:
# Look at APPLICATION_TYPE value counts for binning
application_counts = application_df['APPLICATION_TYPE'].value_counts()
application_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [30]:
application_types_to_replace = list(application_counts[application_counts < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [31]:
# Look at CLASSIFICATION value counts for binning
class_counts = application_df['CLASSIFICATION'].value_counts()
class_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [32]:
# Determine which values to replace if counts are less than 1000
classes_to_replace = list(class_counts[class_counts < 1000].index)

# Replace in dataframe
for cls in classes_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [33]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

In [34]:
application_with_dummies_df = pd.get_dummies(application_df)

In [35]:
# Split our preprocessed data into our features and target arrays
X = application_with_dummies_df.drop(["IS_SUCCESSFUL"], axis='columns').values
y = application_with_dummies_df["IS_SUCCESSFUL"].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [36]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [37]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  100
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               39900     
                                                                 
 dense_1 (Dense)             (None, 30)                3030      
                                                                 
 dense_2 (Dense)             (None, 10)                310       
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 43251 (168.95 KB)
Trainable params: 43251 (168.95 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [38]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [39]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100


804/804 [==============================] - 1s 921us/step - loss: 0.5054 - accuracy: 0.7739
Epoch 2/100
804/804 [==============================] - 1s 887us/step - loss: 0.4460 - accuracy: 0.7933
Epoch 3/100
804/804 [==============================] - 1s 897us/step - loss: 0.4358 - accuracy: 0.7960
Epoch 4/100
804/804 [==============================] - 1s 873us/step - loss: 0.4306 - accuracy: 0.7960
Epoch 5/100
804/804 [==============================] - 1s 902us/step - loss: 0.4281 - accuracy: 0.7974
Epoch 6/100
804/804 [==============================] - 1s 917us/step - loss: 0.4255 - accuracy: 0.7973
Epoch 7/100
804/804 [==============================] - 1s 865us/step - loss: 0.4240 - accuracy: 0.7990
Epoch 8/100
804/804 [==============================] - 1s 919us/step - loss: 0.4227 - accuracy: 0.7999
Epoch 9/100
804/804 [==============================] - 1s 913us/step - loss: 0.4208 - accuracy: 0.7995
Epoch 10/100
804/804 [==============================] - 1s 847us/step -

804/804 [==============================] - 1s 839us/step - loss: 0.4028 - accuracy: 0.8076
Epoch 76/100
804/804 [==============================] - 1s 826us/step - loss: 0.4029 - accuracy: 0.8081
Epoch 77/100
804/804 [==============================] - 1s 821us/step - loss: 0.4027 - accuracy: 0.8082
Epoch 78/100
804/804 [==============================] - 1s 826us/step - loss: 0.4027 - accuracy: 0.8085
Epoch 79/100
804/804 [==============================] - 1s 825us/step - loss: 0.4027 - accuracy: 0.8077
Epoch 80/100
804/804 [==============================] - 1s 814us/step - loss: 0.4022 - accuracy: 0.8078
Epoch 81/100
804/804 [==============================] - 1s 825us/step - loss: 0.4026 - accuracy: 0.8085
Epoch 82/100
804/804 [==============================] - 1s 822us/step - loss: 0.4025 - accuracy: 0.8076
Epoch 83/100
804/804 [==============================] - 1s 829us/step - loss: 0.4027 - accuracy: 0.8081
Epoch 84/100
804/804 [==============================] - 1s 817us/step - loss:

In [40]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.4456 - accuracy: 0.7900 - 256ms/epoch - 956us/step
Loss: 0.445619136095047, Accuracy: 0.789970874786377


In [41]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")

C:\Users\thong\anaconda3\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Compile, Train and Evaluate the Model

In [42]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_dummies = pd.get_dummies(application_df)

In [43]:
# Split our preprocessed data into our features and target arrays
X = application_df_dummies.drop(["IS_SUCCESSFUL"], axis='columns').values
y = application_df_dummies["IS_SUCCESSFUL"].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [44]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [45]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  100
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 100)               39900     
                                                                 
 dense_5 (Dense)             (None, 30)                3030      
                                                                 
 dense_6 (Dense)             (None, 1)                 31        
                                                                 
Total params: 42961 (167.82 KB)
Trainable params: 42961 (167.82 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [46]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [47]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 822us/step - loss: 0.4720 - accuracy: 0.7760
Epoch 2/100
804/804 [==============================] - 1s 862us/step - loss: 0.4361 - accuracy: 0.7924
Epoch 3/100
804/804 [==============================] - 1s 834us/step - loss: 0.4302 - accuracy: 0.7947
Epoch 4/100
804/804 [==============================] - 1s 834us/step - loss: 0.4286 - accuracy: 0.7959
Epoch 5/100
804/804 [==============================] - 1s 809us/step - loss: 0.4261 - accuracy: 0.7951
Epoch 6/100
804/804 [==============================] - 1s 792us/step - loss: 0.4239 - accuracy: 0.7961
Epoch 7/100
804/804 [==============================] - 1s 794us/step - loss: 0.4233 - accuracy: 0.7969
Epoch 8/100
804/804 [==============================] - 1s 792us/step - loss: 0.4221 - accuracy: 0.7992
Epoch 9/100
804/804 [==============================] - 1s 801us/step - loss: 0.4210 - accuracy: 0.7996
Epoch 10/100
804/804 [==============================] - 1s 800us/step - l

804/804 [==============================] - 1s 819us/step - loss: 0.4027 - accuracy: 0.8071
Epoch 80/100
804/804 [==============================] - 1s 784us/step - loss: 0.4023 - accuracy: 0.8073
Epoch 81/100
804/804 [==============================] - 1s 793us/step - loss: 0.4029 - accuracy: 0.8081
Epoch 82/100
804/804 [==============================] - 1s 786us/step - loss: 0.4027 - accuracy: 0.8076
Epoch 83/100
804/804 [==============================] - 1s 787us/step - loss: 0.4028 - accuracy: 0.8081
Epoch 84/100
804/804 [==============================] - 1s 786us/step - loss: 0.4022 - accuracy: 0.8077
Epoch 85/100
804/804 [==============================] - 1s 796us/step - loss: 0.4029 - accuracy: 0.8078
Epoch 86/100
804/804 [==============================] - 1s 783us/step - loss: 0.4022 - accuracy: 0.8076
Epoch 87/100
804/804 [==============================] - 1s 785us/step - loss: 0.4021 - accuracy: 0.8077
Epoch 88/100
804/804 [==============================] - 1s 787us/step - loss:

In [48]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.4695 - accuracy: 0.7908 - 219ms/epoch - 817us/step
Loss: 0.46952179074287415, Accuracy: 0.7907871603965759


In [49]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")

C:\Users\thong\anaconda3\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
